<a href="https://colab.research.google.com/github/Siera-Collab/Tugas-KI-Math-UI-Siera-Barokatillah/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Minggu ke-7**

Nama : Siera Barokatillah


NPM : 2006568714

Souce of model : https://huggingface.co/models?search=sentiment%20analysis

In [25]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='distilroberta-base')
unmasker("The man worked as a <mask>.")
unmasker("The woman worked as a <mask>.")


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transforme

[{'score': 0.2160816788673401,
  'token': 35698,
  'token_str': ' waitress',
  'sequence': 'The woman worked as a waitress.'},
 {'score': 0.07409851253032684,
  'token': 38233,
  'token_str': ' waiter',
  'sequence': 'The woman worked as a waiter.'},
 {'score': 0.06863326579332352,
  'token': 33080,
  'token_str': ' bartender',
  'sequence': 'The woman worked as a bartender.'},
 {'score': 0.05877527594566345,
  'token': 9008,
  'token_str': ' nurse',
  'sequence': 'The woman worked as a nurse.'},
 {'score': 0.05221009626984596,
  'token': 29754,
  'token_str': ' maid',
  'sequence': 'The woman worked as a maid.'}]

In [26]:
from transformers import DistilBertTokenizer, TFDistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model dan tokenizer
tokenizer = AutoTokenizer.from_pretrained("ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa")
model = AutoModelForSequenceClassification.from_pretrained("ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa")

# Contoh kalimat untuk prediksi
inputs = tokenizer("Saya senang belajar pemrograman.", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

# Mendapatkan hasil prediksi
predicted_class_id = logits.argmax().item()
sentiment = model.config.id2label[predicted_class_id]
print(f"Sentimen: {sentiment}")


Sentimen: POSITIVE


In [28]:
print(model.config.id2label)


{0: 'POSITIVE', 1: 'NEUTRAL', 2: 'NEGATIVE'}


**Check the model’s accuracy using a synthetic dataset**


In [29]:
import pandas as pd

# Dataset sintetis dengan label yang sesuai dengan model
data = {
    "text": [
        "Saya sangat senang belajar!",       # POSITIVE
        "Ini adalah hari yang buruk.",       # NEGATIVE
        "Saya tidak peduli tentang hal ini." # NEUTRAL
    ],
    "label": ["POSITIVE", "NEGATIVE", "NEUTRAL"]
}

# Konversi ke DataFrame
df = pd.DataFrame(data)

# Prediksi setiap teks dalam dataset sintetis
predictions = []
for text in df["text"]:
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    sentiment = model.config.id2label[predicted_class_id]
    predictions.append(sentiment)

# Tambahkan prediksi ke DataFrame
df["predicted_label"] = predictions

# Hitung akurasi
accuracy = (df["label"] == df["predicted_label"]).mean()
print(f"Akurasinya adalah: {accuracy * 100:.2f}%")


Akurasinya adalah: 66.67%


**Implement attention transformer on that model**

In [30]:
from transformers import AutoModelForSequenceClassification
import torch

# Load ulang model dengan output layer perhatian diaktifkan
model = AutoModelForSequenceClassification.from_pretrained("ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa", output_attentions=True)

# Contoh kalimat
inputs = tokenizer("Saya senang belajar pemrograman.", return_tensors="pt")
outputs = model(**inputs)

# Ambil layer perhatian
attentions = outputs.attentions
print(f"Jumlah layer perhatian: {len(attentions)}")
print(f"Bentuk data layer perhatian: {attentions[0].shape}")


Jumlah layer perhatian: 12
Bentuk data layer perhatian: torch.Size([1, 12, 7, 7])


**Check the model accuracy after using attention transformer using the same synthetic dataset and compare**

In [31]:
from transformers import AutoModelForSequenceClassification
import torch

# Load ulang model dengan output layer perhatian diaktifkan
model_with_attention = AutoModelForSequenceClassification.from_pretrained(
    "ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa",
    output_attentions=True
)

# Prediksi setiap teks dalam dataset sintetis dengan model baru
new_predictions = []
for text in df["text"]:
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model_with_attention(**inputs)
        logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    sentiment = model_with_attention.config.id2label[predicted_class_id]
    new_predictions.append(sentiment)

# Tambahkan prediksi baru ke DataFrame
df["new_predicted_label"] = new_predictions

# Hitung akurasi baru
new_accuracy = (df["label"] == df["new_predicted_label"]).mean()
print(f"Akurasinya setelah analisis perhatian adalah: {new_accuracy * 100:.2f}%")


Akurasinya setelah analisis perhatian adalah: 66.67%


Catatan: Mengaktifkan lapisan perhatian (output_attentions=True) sendiri tidak akan meningkatkan akurasi kecuali model tersebut dilatih kembali (fine-tuning) dengan memanfaatkan lapisan perhatian.